# Genetic Algorithm search for continuous function optimization


In [1]:
import random
from numpy.random import randint
from numpy.random import rand
import numpy as np
from numpy import *
from numpy.random import randn
from numpy.random import seed
from matplotlib import pyplot
from scipy import stats
from scipy.optimize import differential_evolution
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

#### A simple x^2 minimization function is used in this first part to check how the implementation works

In [2]:
# a simple objective function :
# x^2 minimization function that takes input variables and has an optima at  f(0, 0) = 0.0.
def objective(x):
    return x[0]**2.0 + x[1]**2.0

In [3]:
# Decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract substring
        start = i * n_bits
        end = (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # append
        decoded.append(value)
    return decoded

### Tournament selection

In [4]:
# Tournament selection
def ts_selection(population, fitnesses, k=3):
    # first random selection
    selection_ix = randint(len(population))
    for ix in randint(0, len(population), k-1):
        #  perform a tournament
        if fitnesses[ix] < fitnesses[selection_ix]:
            selection_ix = ix
    return population[selection_ix]

### Roulette wheel selection

In [5]:
# Roulette wheel selection
def roulette_selection(population, fitnesses, num=2):
    total_fitness = float(sum(fitnesses))
    rel_fitness = [f/total_fitness for f in fitnesses]
    # Generate probability intervals for each individual
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    
    # Draw new population
    for n in range(num):
        r = rand()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:          
                selected = individual
                break
         
    return selected

### Linear rank selection

In [6]:
# Linear rank selection
def lr_selection(population,fitnesses, num=2):
    # Number of individuals in population.
    n = len(fitnesses)
    # Gauss formula to get the sum of all ranks
    rank_sum = n * (n + 1) / 2

    probs = []
    
    # Sort and go through all individual fitnesses and get selection probability
    for (rank, individual) in enumerate(sorted(population), 1):
        probability = (float(rank) / rank_sum) * 100
        probs.append(probability)

    for n in range(num):
        r = rand()
        for (rank, individual) in enumerate(sorted(population), 1):
            if r <= probs[rank]:
                selected = individual
                break      
    return selected

### Crossover

In [7]:
# Crossover 
def crossover(parent1, parent2, r_cross):
    # children
    child1, child2 = parent1.copy(), parent2.copy()
    
    if rand() < r_cross:
        # select crossover point not on the end of the string
        point = randint(1, len(parent1)-2)
        # do crossover
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
    return [child1, child2]

### Mutation

In [8]:
# Mutation 
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
    # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

### Genetic Algorithm

In [9]:
# Genetic Algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut, selection_method):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        fitnesses = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if fitnesses[i] < best_eval:
                best, best_eval = pop[i], fitnesses[i]
                print(">%d f(%s) = %f" % (gen,  decoded[i], fitnesses[i]))
                
        # -SELECTION METHODS-       
        # Tournament selection
        if(selection_method == "ts_selection"): 
            selected = [ts_selection(pop, fitnesses) for _ in range(n_pop)]
        # Roulette selection
        if(selection_method == "roulette_selection"):
            selected = [roulette_selection(pop, fitnesses) for _ in range(n_pop)]
        # Linear rank selection
        if(selection_method == "lr_selection"):
            selected = [lr_selection(pop, fitnesses) for _ in range(n_pop)]
            
        
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)               
            
        # replace population
        pop = children
        
    return [best, best_eval]

#### When running the GA, uncomment the selection method that you want to run

In [10]:
# Run the GA

# define range for each input variable
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# number of iterations
n_iter = 100
# bits per input variable: The bit string will have 32 bits given two input variables
n_bits = 16
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))

# Choose the selection methods:
#selection_method = "ts_selection"
#selection_method = "roulette_selection"
selection_method = "lr_selection"

# run the genetic algorithm 
best, fitness = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut, selection_method)
decoded = decode(bounds, n_bits, best)
print('Best: f(%s) = %f' % (decoded, fitness))


>0 f([0.568084716796875, 4.13604736328125]) = 17.429608
>0 f([-3.817901611328125, 0.10345458984375]) = 14.587076
>0 f([1.68487548828125, 1.63787841796875]) = 5.521451
>0 f([-0.11199951171875, 1.723480224609375]) = 2.982928
>0 f([0.951385498046875, 0.481719970703125]) = 1.137188
>0 f([-0.67962646484375, -0.036468505859375]) = 0.463222
>0 f([0.167083740234375, 0.619659423828125]) = 0.411895
>1 f([-0.11199951171875, 0.41900634765625]) = 0.188110
>6 f([0.02899169921875, 0.30517578125]) = 0.093973
>6 f([0.02105712890625, 0.26702880859375]) = 0.071748
>51 f([0.0, 0.06103515625]) = 0.003725
>61 f([0.0006103515625, 0.030517578125]) = 0.000932
>92 f([0.0, 0.001220703125]) = 0.000001
Best: f([0.0, 0.001220703125]) = 0.000001


#### For comparison, Stochasitc Hill Climbing, Random Search, Evolution Strategy, and Differential Evolution algorithm has been implemented

### Stochastic hill climbing

In [11]:
# Stochastic hill climbing local search algorithm

# check if within the bounds of the search
def in_bounds(point, bounds):
    for d in range(len(bounds)):
        # check if out of bounds for this dimension
        if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
            return False
    return True

# hill climbing local search algorithm
def hillclimbing(objective, bounds, n_iterations, step_size):
    # generate an initial point
    solution = None
    while solution is None or not in_bounds(solution, bounds):
        # generate random point in the search space
        solution = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
    
    # evaluate the initial point
    solution_eval = objective(solution)
    # run hill climbing
    for i in range(n_iterations):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            # generate a version of the current working solution
            candidate = solution + randn(len(bounds)) * step_size
        # evaluate candidate point
        candidate_eval = objective(candidate)
        # check if the new point should be kept
        if candidate_eval <= solution_eval:
            # store new point
            solution, solution_eval = candidate, candidate_eval
            print('>%d f(%s) = %.5f' % (i, solution, solution_eval))
    return [solution, solution_eval]

In [12]:
# Perform the hill climbing search

# seed the pseudorandom number generator
seed(1)
# range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# the total iterations
n_iter = 1000
# the maximum step size
step_size = 0.05

# run the hill climbing search
best, score = hillclimbing(objective, bounds, n_iter, step_size)
print('Best: f(%s) = %f' % (best, score))


>0 f([-0.85618854  2.1495965 ]) = 5.35382
>1 f([-0.81291816  2.03451957]) = 4.80011
>2 f([-0.72567757  1.99645922]) = 4.51246
>3 f([-0.70972562  1.9839907 ]) = 4.43993
>4 f([-0.63662022  1.88098367]) = 3.94338
>5 f([-0.65274108  1.86178095]) = 3.89230
>6 f([-0.59605261  1.80678639]) = 3.61976
>7 f([-0.60467402  1.76289347]) = 3.47342
>11 f([-0.55963122  1.72870707]) = 3.30162
>12 f([-0.56577573  1.6819186 ]) = 3.14895
>14 f([-0.60035877  1.66208093]) = 3.12294
>15 f([-0.6347174   1.61982064]) = 3.02669
>19 f([-0.64430918  1.5754392 ]) = 2.89714
>21 f([-0.64176879  1.54358941]) = 2.79454
>24 f([-0.62676028  1.52597692]) = 2.72143
>30 f([-0.60111379  1.51107228]) = 2.64468
>31 f([-0.57668788  1.50729369]) = 2.60450
>33 f([-0.46740911  1.43746888]) = 2.28479
>36 f([-0.45162736  1.33635882]) = 1.98982
>39 f([-0.46274377  1.32632091]) = 1.97326
>44 f([-0.40279788  1.33557873]) = 1.94602
>45 f([-0.42156212  1.30364221]) = 1.87720
>46 f([-0.4003874   1.30750922]) = 1.86989
>52 f([-0.36335958 

### Random search

In [13]:
# random search for function optimization

def random_search(objective, bounds):
    best_solution = None

    # get the best solution
    for i in range(100):
        # generate a random point in the search space
        sample = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        # evaluate
        sample_eval = objective(sample)

        # candidate
        candidate = sample + randn(len(bounds)) * (bounds[:, 1] - bounds[:, 0])   
        # evaluate candidate
        candidate_eval = objective(candidate)

        if candidate_eval < sample_eval:            
            solution, solution_eval = candidate, candidate_eval
            print('>%d f(%s) = %.5f' % (i, solution, solution_eval))
            
    return [solution, solution_eval]

In [14]:
# Run random search

bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])  
best, score = random_search(objective, bounds)

# best solution
print('Best: f(%s) = %.5f' %(best, score))

>24 f([-0.5021013   2.28326656]) = 5.46541
>40 f([-0.95856209 -1.53776796]) = 3.28357
>44 f([-1.74948709 -1.70899921]) = 5.98138
>48 f([2.8455663  2.58534297]) = 14.78125
>64 f([ 1.67400781 -2.89321445]) = 11.17299
>65 f([-1.76051458 -3.21228685]) = 13.41820
>84 f([ 0.05445522 -3.84101358]) = 14.75635
>90 f([-1.38829057  0.05433623]) = 1.93030
Best: f([-1.38829057  0.05433623]) = 1.93030


### (mu, lambda) Evolution Strategy

In [15]:
# check if within the bounds of the search
def in_bounds(point, bounds):
    for d in range(len(bounds)):
        # check if out of bounds for dimension
        if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
            return False
    return True
 
# Evolution Strategy (mu, lambda) algorithm
def evolution_strategy(objective, bounds, n_iter, step_size, mu, lam):
    best, best_eval = None, 1e+10
    # calculate the number of children per parent
    n_children = int(lam / mu)
    # initial population
    population = list()
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        population.append(candidate)
    # perform the search
    for epoch in range(n_iter):
        # evaluate fitness for the population
        fitness = [objective(c) for c in population]
        # rank scores in ascending order
        ranks = argsort(argsort(fitness))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
        # create children from parents
        children = list()
        for i in selected:
            # check if this parent is the best solution ever seen
            if fitness[i] < best_eval:
                best, best_eval = population[i], fitness[i]
                print('%d, f(%s) = %.5f' % (epoch, best, best_eval))
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child, bounds):
                    child = population[i] + randn(len(bounds)) * step_size
                children.append(child)
        # replace population with children
        population = children
    return [best, best_eval]

In [16]:
# Run evolution strategy

# seed the pseudorandom number generator
seed(1)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# the maximum step size
step_size = 0.15
# number of parents selected
mu = 20
# the total iterations
n_iter = 1000
# the number of children generated by parents
lam = 100
# run the evolution strategy algorithm
best, score = evolution_strategy(objective, bounds, n_iter, step_size, mu, lam)
print('Best: f(%s) = %f' % (best, score))

0, f([-0.82977995  2.20324493]) = 5.54282
0, f([-1.03232526  0.38816734]) = 1.21637
0, f([-0.82695198  0.58689828]) = 1.02830
0, f([-0.50087867  0.78389614]) = 0.86537
1, f([-0.62723341  0.5438021 ]) = 0.68914
1, f([-0.43644974  0.69284638]) = 0.67052
1, f([-0.48494526  0.5550441 ]) = 0.54325
2, f([-0.67433117  0.22584107]) = 0.50573
2, f([-0.46090412  0.52983017]) = 0.49315
2, f([-0.35429979  0.47877553]) = 0.35475
3, f([-0.54633885  0.20488431]) = 0.34046
3, f([-0.46638403  0.30806051]) = 0.31242
3, f([-0.34390005  0.43541393]) = 0.30785
3, f([-0.51386116  0.12542443]) = 0.27978
3, f([-0.37653137  0.3163851 ]) = 0.24188
3, f([-0.08036353  0.32847434]) = 0.11435
4, f([-0.31545779  0.03257867]) = 0.10057
4, f([-0.22234445  0.21645456]) = 0.09629
4, f([-0.17856454  0.25097779]) = 0.09488
4, f([-0.15533927  0.16223786]) = 0.05045
5, f([-0.16426282  0.09142891]) = 0.03534
5, f([-0.05815055 -0.04050955]) = 0.00502
6, f([0.02583607 0.0520299 ]) = 0.00337
6, f([-0.00846378 -0.02112442]) = 0.

## Differential evolution algorithm

In [17]:
# Differential Evolution 

# the bounds of the search
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

# run the differential evolution search (imported from scipy.optimize)
result = differential_evolution(objective, bounds)
# summarize the result
print('Number of Evaluations: %d' % result['nfev'])
# evaluate solution
solution = result['x']
evaluation = objective(solution)
print('Best: f(%s) = %.5f' % (solution, evaluation))


Number Evaluations: 2913
Best: f([0. 0.]) = 0.00000


---------------------------------------

# Benchmarks

#### Six benchmarks will be used to compare the algorithms (Ackley, Bohachevsky, Schaffer, Egg-holder, Schwefel, Drop-wave)

## Ackley multimodal function

In [18]:
# ackley multimodal objective function
def ackley_function(v):
    x, y = v
    return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20 

#### Re-run algorithms
This runs all the algorithms implemented above (GA, Stochastic Hill Climber, Random Search, Evolution Strategy, and Differential Evolution algorithm) on the given objective function.

Uncomment the wanted selection operator to run on the GA

In [19]:
def run_algorithms(objective):
    print("[",objective,"]\n")
    
    # ----Run genetic algorithm
  
    # the range for each input variable
    bounds = [[-5.0, 5.0], [-5.0, 5.0]]
    # the total iterations
    n_iter = 100
    # bits per input variable
    n_bits = 16
    # population size
    n_pop = 100
    # crossover rate
    r_cross = 0.9
    # mutation rate
    r_mut = 1.0 / (float(n_bits) * len(bounds))
    
    #-Choose the selection methods-
    selection_method = "ts_selection"
    #selection_method = "roulette_selection"
    #selection_method = "lr_selection"
    
    print("----Genetic algorithm running----")
    best, fitness = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut,selection_method)
    decoded = decode(bounds, n_bits, best)
    print('Best: f(%s) = %f' % (decoded, fitness))


    # ----Run stochastic hill climbing local search

    seed(1)
    # range for input
    bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
    # maximum step size
    step_size = 0.05
    # total iterations
    n_iter = 1000
 
    print("\n----Stochastic hill climbing running----")
    best, score = hillclimbing(objective, bounds, n_iter, step_size)
    print('Best: f(%s) = %f' % (best, score))

    # ---- Run Random search

    bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])  
    print("\n----Random search running----")
    best, score = random_search(objective, bounds)
    # best solution
    print('Best: f(%s) = %.5f' %(best, score))

    # ---- Run evolution strategy

    seed(1)
    bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
    n_iter = 5000
    # maximum step size
    step_size = 0.15
    # number of parents selected
    mu = 20
    # the number of children generated by parents
    lam = 100   
    print("\n----Evolution stratey running----")
    best, score = evolution_strategy(objective, bounds, n_iter, step_size, mu, lam)
    print('Best: f(%s) = %f' % (best, score))

    # ----Differential evolution global optimization

    bounds = [[-5.0, 5.0], [-5.0, 5.0]]
    result = differential_evolution(objective, bounds)
    print("\n----Differential evolution global optimization running----")
    # summarize the result
    print('Total Evaluations: %d' % result['nfev'])
    # evaluate solution
    solution = result['x']
    evaluation = objective(solution)
    print('Best: f(%s) = %.5f' % (solution, evaluation))    

In [20]:
run_algorithms(ackley_function)
 

[ <function ackley_function at 0x0000023AF71E9940> ]

----Genetic algorithm running----
>0 f([0.749664306640625, -3.593292236328125]) = 10.158646
>0 f([-0.503692626953125, 0.52734375]) = 4.307454
>0 f([0.86669921875, 1.10626220703125]) = 4.253672
>0 f([-0.07232666015625, 0.54962158203125]) = 3.252650
>1 f([-0.07232666015625, 0.848541259765625]) = 2.892890
>1 f([0.402679443359375, 0.010528564453125]) = 2.732140
>2 f([-0.072174072265625, 0.010528564453125]) = 0.342024
>3 f([-0.072174072265625, 0.009918212890625]) = 0.341466
>4 f([-0.072174072265625, 0.0054931640625]) = 0.338414
>5 f([-0.071563720703125, 0.0054931640625]) = 0.334548
>5 f([-0.06683349609375, 0.01800537109375]) = 0.318680
>6 f([0.04974365234375, 0.01068115234375]) = 0.211427
>6 f([-0.049591064453125, 0.0054931640625]) = 0.206081
>7 f([-0.048980712890625, 0.0054931640625]) = 0.202826
>7 f([-0.03875732421875, 0.0299072265625]) = 0.201266
>8 f([-0.03875732421875, 0.029449462890625]) = 0.199776
>8 f([-0.02777099609375, 0.010223

## Bohachevsky function

In [21]:
def bohachevsky_function(v):
    x1, x2 = v
    return x1**2 +2*(x2**2)-0.3*cos(3*pi*x1)-0.4*cos(4*pi*x2)+0.7

#### Run algorithms

In [22]:
run_algorithms(bohachevsky_function)

[ <function bohachevsky_function at 0x0000023AF72035E0> ]

----Genetic algorithm running----
>0 f([-1.00006103515625, 1.377716064453125]) = 5.782675
>0 f([0.54412841796875, 0.990142822265625]) = 2.438702
>0 f([-0.248870849609375, 0.04913330078125]) = 0.650479
>1 f([0.549468994140625, 0.04913330078125]) = 0.545738
>2 f([0.532379150390625, -0.0238037109375]) = 0.512188
>3 f([-0.09185791015625, -0.1116943359375]) = 0.472396
>4 f([-0.092010498046875, 0.06134033203125]) = 0.234963
>4 f([-0.082855224609375, 0.0506591796875]) = 0.177264
>5 f([-0.090179443359375, 0.030670166015625]) = 0.141344
>5 f([-0.092620849609375, 0.0042724609375]) = 0.116416
>6 f([-0.069427490234375, 0.0128173828125]) = 0.072291
>6 f([-0.013885498046875, 0.031585693359375]) = 0.035851
>7 f([-0.013885498046875, 0.030975341796875]) = 0.034599
>7 f([-0.010223388671875, 0.003509521484375]) = 0.001910
>10 f([-0.010223388671875, 0.00274658203125]) = 0.001749
>10 f([-0.004119873046875, 0.003509521484375]) = 0.000657
>12 f([-0.0

## Schaffer function

In [23]:
def schaffer_function(v):
    x, y = v
    num = (np.sin((x**2 + y**2)**2)**2) - 0.5
    den = (1 + 0.001*(x**2 + y**2))**2 
    return 0.5 + num/den

#### Run algorithms

In [24]:
run_algorithms(schaffer_function)

[ <function schaffer_function at 0x0000023AF7135AF0> ]

----Genetic algorithm running----
>0 f([-2.059478759765625, 0.676727294921875]) = 0.013271
>0 f([-0.06622314453125, 0.13885498046875]) = 0.000024
>1 f([-0.06622314453125, 0.135345458984375]) = 0.000023
>2 f([-0.034942626953125, -0.01434326171875]) = 0.000001
>11 f([-0.00885009765625, -0.023651123046875]) = 0.000001
>12 f([-0.009765625, -0.005340576171875]) = 0.000000
>15 f([-0.00946044921875, -0.001983642578125]) = 0.000000
>17 f([-0.00213623046875, 0.000457763671875]) = 0.000000
>23 f([-0.00091552734375, -0.0018310546875]) = 0.000000
>25 f([-0.00091552734375, -0.001678466796875]) = 0.000000
>25 f([-0.00091552734375, -0.0006103515625]) = 0.000000
>31 f([-0.000762939453125, -0.000762939453125]) = 0.000000
>35 f([-0.00091552734375, -0.000457763671875]) = 0.000000
>36 f([-0.000152587890625, -0.000762939453125]) = 0.000000
>39 f([-0.00030517578125, -0.000457763671875]) = 0.000000
>40 f([-0.00030517578125, -0.000152587890625]) = 0.0000

## Egg-Holder function

In [25]:
def eggholder_function(v):
    x1, x2 = v
    a=sqrt(fabs(x2+x1/2+47))
    b=sqrt(fabs(x1-(x2+47)))
    c=-(x2+47)*sin(a)-x1*sin(b)
    return c

#### Run algorithms

In [26]:
run_algorithms(eggholder_function)

[ <function eggholder_function at 0x0000023AF71E9DC0> ]

----Genetic algorithm running----
>0 f([4.31304931640625, -1.705780029296875]) = -26.281264
>0 f([1.950531005859375, 0.217437744140625]) = -29.748590
>0 f([4.858551025390625, 4.36614990234375]) = -47.073884
>1 f([4.857330322265625, 4.762115478515625]) = -48.212152
>3 f([4.8974609375, 4.920806884765625]) = -48.703628
>4 f([4.85931396484375, 4.96429443359375]) = -48.785392
>5 f([4.90081787109375, 4.96429443359375]) = -48.829362
>6 f([4.93438720703125, 4.969024658203125]) = -48.877946
>6 f([4.90081787109375, 4.986419677734375]) = -48.891396
>7 f([4.93499755859375, 4.97406005859375]) = -48.892704
>7 f([4.93743896484375, 4.97406005859375]) = -48.895263
>8 f([4.936676025390625, 4.98321533203125]) = -48.920121
>8 f([4.93743896484375, 4.99847412109375]) = -48.963648
>9 f([4.996490478515625, 4.998779296875]) = -49.026015
>21 f([4.998931884765625, 4.99847412109375]) = -49.027690
>24 f([4.998779296875, 4.99908447265625]) = -49.029239
>24 f(

12, f([4.99846728 4.99219761]) = -49.00965
12, f([4.99243075 4.9981673 ]) = -49.02010
98, f([4.99963624 4.99647067]) = -49.02281
99, f([4.99905396 4.99691923]) = -49.02347
142, f([4.99155485 4.99991991]) = -49.02409
143, f([4.99651955 4.99975288]) = -49.02877
238, f([4.99935348 4.9987719 ]) = -49.02896
549, f([4.99684388 4.99974181]) = -49.02907
1196, f([4.99641678 4.99997787]) = -49.02929
1254, f([4.99899874 4.99988932]) = -49.03172
3039, f([4.99888115 4.99999354]) = -49.03189
3161, f([4.99977118 4.99989105]) = -49.03252
Best: f([4.99977118 4.99989105]) = -49.032523

----Differential evolution global optimization running----
Total Evaluations: 426
Best: f([5. 5.]) = -49.03306


## Schwefel function

In [27]:
def schwefel_function(v):  
    x1,x2 = v
    return 418.9829*2 - x1 * sin( sqrt( abs( x1 )))-x2*sin(sqrt(abs(x2)))

#### Run algorithms

In [28]:
run_algorithms(schwefel_function)

[ <function schwefel_function at 0x0000023AF71E9B80> ]

----Genetic algorithm running----
>0 f([1.735687255859375, 3.874053955078125]) = 832.713463
>0 f([2.8179931640625, 4.89013671875]) = 831.243497
>0 f([4.686737060546875, 3.41766357421875]) = 830.795555
>1 f([4.477386474609375, 3.874053955078125]) = 830.565447
>2 f([3.926239013671875, 4.46746826171875]) = 830.540783
>2 f([4.54193115234375, 4.186553955078125]) = 830.396076
>3 f([4.3267822265625, 4.674224853515625]) = 830.307255
>4 f([4.24041748046875, 4.9517822265625]) = 830.292596
>4 f([4.627685546875, 4.94232177734375]) = 830.168451
>5 f([4.68505859375, 4.892578125]) = 830.161435
>6 f([4.67864990234375, 4.9517822265625]) = 830.155296
>6 f([4.979248046875, 4.9420166015625]) = 830.106681
>8 f([4.979248046875, 4.968719482421875]) = 830.103620
>8 f([4.97314453125, 4.988250732421875]) = 830.102212
>9 f([4.9969482421875, 4.9810791015625]) = 830.100506
>10 f([4.9969482421875, 4.981231689453125]) = 830.100490
>12 f([4.997711181640625, 4.98

6, f([4.83346274 4.91698576]) = 830.12941
6, f([4.89649864 4.88104247]) = 830.12482
6, f([4.90189567 4.99884877]) = 830.10984
7, f([4.95365888 4.97636035]) = 830.10562
7, f([4.97346774 4.99290174]) = 830.10171
9, f([4.999753   4.97142727]) = 830.10126
12, f([4.99835656 4.99484907]) = 830.09897
24, f([4.99898783 4.99590594]) = 830.09881
65, f([4.99598883 4.99936134]) = 830.09876
147, f([4.99781921 4.99841993]) = 830.09867
176, f([4.99993106 4.99690662]) = 830.09862
358, f([4.99952385 4.99993718]) = 830.09836
965, f([4.99961724 4.99986886]) = 830.09836
1425, f([4.99964065 4.99995961]) = 830.09835
Best: f([4.99964065 4.99995961]) = 830.098347

----Differential evolution global optimization running----
Total Evaluations: 69
Best: f([5. 5.]) = 830.09831


## Drop-wave function

In [29]:
def dropwave_function(v):
    x1, x2 = v
    b=0.5*(x1*x1+x2*x2)+2
    a=-(1+cos(12*sqrt(x1*x1+x2*x2)))/b
    return a

#### Run algorithms

In [30]:
run_algorithms(dropwave_function)

[ <function dropwave_function at 0x0000023AF71E9E50> ]

----Genetic algorithm running----
>0 f([-2.209625244140625, -1.609649658203125]) = -0.205809
>0 f([-1.425323486328125, -2.13653564453125]) = -0.344954
>0 f([-0.28228759765625, -0.46844482421875]) = -0.912315
>2 f([-0.28228759765625, -0.45684814453125]) = -0.926709
>4 f([-0.282135009765625, -0.43121337890625]) = -0.935429
>4 f([0.2618408203125, 0.45166015625]) = -0.936129
>5 f([-0.282135009765625, -0.43609619140625]) = -0.936223
>13 f([0.054473876953125, -0.516815185546875]) = -0.936236
>15 f([0.243988037109375, -0.458984375]) = -0.936240
>20 f([0.11077880859375, -0.5084228515625]) = -0.936245
>26 f([0.1055908203125, -0.509490966796875]) = -0.936245
>26 f([0.079498291015625, -0.514068603515625]) = -0.936245
>37 f([0.1043701171875, -0.5096435546875]) = -0.936245
>46 f([0.10284423828125, -0.50994873046875]) = -0.936245
>47 f([0.101318359375, -0.51025390625]) = -0.936245
Best: f([0.101318359375, -0.51025390625]) = -0.936245

----Stoch

-----------------------------------